In [1]:
import gym
from gym import wrappers
import qlearning
import numpy as np
import matplotlib.pyplot as plt

NUM_EPISODES = 20000
N_BINS = [8, 8, 8, 8]
MAX_STEPS = 200
FAIL_PENALTY = -1e-6
EPSILON = 0.5
EPSILON_DECAY = 0.99
LEARNING_RATE = 0.05
DISCOUNT_FACTOR = 0.9


RECORD = False

MIN_VALUES = [-0.5, -2.0, -0.5, -3.0]
MAX_VALUES = [0.5, 2.0, 0.5, 3.0]
BINS = [np.linspace(MIN_VALUES[i], MAX_VALUES[i], N_BINS[i])
        for i in range(4)]

In [2]:
import numpy as np

Current_Inventory = 150.
Ships_Status = np.array([0, 0, 0, 0, 0])
Ships_CurrentStatus = np.array([0, 0, 0, 0, 0])
Shipment_Status = np.array([0, 0, 0, 0, 0, 0])

State_No = 0
Shipment_No_Day = 0
OptimizationPeriod = 7

defaultstate = np.concatenate((Current_Inventory, len(Ships_Status), len(Shipment_Status), 
                               Ships_Status, Ships_CurrentStatus, Shipment_Status, 
                               OptimizationPeriod, Shipment_No_Day, State_No), axis = None)

state = np.copy(defaultstate)
action_no = 3


In [3]:
def environment(state, action_no):
    TotalShipmentPerDay = 3
    PurchaseFactor = 0.40
    BCE = 22962.96
    TankBottom = 89
    TankTop = 585
    done = 0
    penalty = -1e07
    Ships_CharterHireRate = np.array([120000, 90000, 45000, 115000, 105000])
    Ships_FuelConsumed = np.array([150, 150, 140, 150, 150])
    Ships_Size = np.array([150, 145, 128, 128, 150])
    Shipment_VoyageDays = np.array([3, 5, 4, 5, 6, 4])
    Shipment_RoundDays = np.array([5, 7, 7, 9, 10, 8])
    Shipment_LNGPrice = np.array([5.8, 6.2, 6.5, 6.3, 6.0, 5.9])
    Shipment_FuelCost = np.array([140, 130, 150, 145, 140, 142])
    Shipment_Size = np.array([2, 2, 2, 2, 2, 2])
    action_space = np.matrix([0, 0])
    for i in range(int(state[1])):
        for j in range(int(state[2])):
            action_space = np.vstack([action_space, (i+1, j+1)])
    action = np.ravel(action_space[action_no])
    Production = np.array([182, 0, 0, 175, 0, 0, 165, 0, 0, 175, 0, 0, 
                           175, 0, 0, 175, 0, 0, 175, 0, 0, 175, 0, 0,
                           175, 0, 0, 175, 0, 0, 175, 0, 0, 175, 0, 0,
                           168, 0, 0, 175, 0, 0, 175, 0, 0, 175, 0, 0,
                           175, 0, 0, 175, 0, 0, 175, 0, 0, 175, 0, 0,
                           175, 0, 0, 175, 0, 0, 175, 0, 0, 175, 0, 0,
                           175, 0, 0, 175, 0, 0, 175, 0, 0, 175, 0, 0,
                           175, 0, 0, 175, 0, 0, 175, 0, 0, 175, 0, 0,
                           175, 0, 0, 175, 0, 0, 175, 0, 0, 175, 0, 0,
                           175, 0, 0, 175, 0, 0, 175, 0, 0, 175, 0, 0])
    Next_State = np.copy(state)
    # Calculate LNG Price of each shipment at each date
#     Next_State[int(3 + Next_State[1]*6 + Next_State[2]*4):int(3 + Next_State[1]*6 + Next_State[2]*5)] = np.array([5.8, 6.4, 6.3, 6.3, 6.0, 5.9])
    # Calculate Fuel Price of each shipment at each date
#     Next_State[int(3 + Next_State[1]*6 + Next_State[2]*5):int(3 + Next_State[1]*6 + Next_State[2]*6)] = np.array([135, 145, 144, 145, 140, 150])
    # Next State
    Next_State[-1] = Next_State[-1]  + 1
    # Shipment Number in the day
    Next_State[-2] = (Next_State[-2] + 1) % TotalShipmentPerDay
    # Adding Production to inventory
    Next_State[0] = Next_State[0] + Production[int(Next_State[-1]-1)]
    # Reducing Current Status by 1
    if Next_State[-2] == 1:
        Next_State[int(3 + Next_State[1]*1):int(3 + Next_State[1]*2)] = \
            [0 if i - 1 < 0 else i - 1 for i in Next_State[int(3 + Next_State[1]*1):int(3 + Next_State[1]*2)]]
    reward = 0
    # Replacing status based on current status
    Next_State[3:int(Next_State[1]+3)] = (Next_State[int(Next_State[1]+3):int(Next_State[1]*2+3)] != 0) + 0
    if Next_State[0] - Ships_Size[int(action[0]-1)] > TankTop:
        reward = penalty + Next_State[-1]
    elif all(i == 0 for i in action):
        reward = 0
    elif Next_State[int(action[0]+3-1)] == 1 or Next_State[int(action[1]+3+Next_State[1]*2-1)] == 1:
        reward = penalty 
        done = 1
    elif Next_State[0] - Ships_Size[int(action[0]-1)] < TankBottom:
        reward = penalty 
        done = 1
    else:
        # Assigning ship as assigned
        Next_State[int(action[0] + 3 - 1)] = 1
        # Assigning current status to Ship
        Next_State[int(action[0] + 3 + Next_State[1] - 1)] = Shipment_RoundDays[int(action[1]-1)]
        # Assigning shipment as assigned
        Next_State[int(action[1] + 3 + Next_State[1]*2 - 1)] = 1
        # Subtracting load size from inventory
        Next_State[0] = Next_State[0] - Ships_Size[int(action[0]-1)]
        # Load Size * LNG Price Minus shipping cost
        reward = Ships_Size[int(action[0]-1)]*Shipment_LNGPrice[int(action[1]-1)]*\
                (1 - PurchaseFactor)*BCE - \
                Ships_FuelConsumed[int(action[0]-1)]*Shipment_RoundDays[int(action[1]-1)]*\
                Shipment_FuelCost[int(action[1]-1)]
    if Next_State[-1] >= Next_State[-3]*3:
        done = 1
    return (Next_State, reward, done)

In [4]:
def discretize(obs):
    return tuple(obs)
#   return tuple([int(np.digitize(obs[i], BINS[i])) for i in range(4)])


def train(agent, environment, history, num_episodes=NUM_EPISODES):
    for i in range(NUM_EPISODES):
        if i % 100 == 0:
            print ("Episode {}".format(i + 1))
        obs = np.copy(defaultstate)
        cur_state = discretize(obs)
#     cur_state = np.copy(obs)
    
    for t in range(MAX_STEPS):
        action = agent.get_action(cur_state)
        observation, reward, done = environment(cur_state, action)
#       next_state = np.copy(observation)
        next_state = discretize(observation)

        if done:
            reward = FAIL_PENALTY
            agent.learn(cur_state, action, next_state, reward, done)
            print("Episode finished after {} timesteps".format(t + 1))
            history.append(t + 1)
            break
        agent.learn(cur_state, action, next_state, reward, done)
        cur_state = discretize(next_state)
        if t == MAX_STEPS - 1:
            history.append(t + 1)
            print("Episode finished after {} timesteps".format(t + 1))
    return agent, history

def get_actions(state):
    return np.array(range(31))

In [5]:
# Ship_Status = defaultstate[3:int(3 + defaultstate[1])] 
# Ship_CurrentStatus = defaultstate[int(3 + defaultstate[1]*1):int(3 + defaultstate[1]*1 + defaultstate[1])] 
# Shipment_Status = defaultstate[int(3 + defaultstate[1]*2):int(3 + defaultstate[1]*2 + defaultstate[2])] 

In [6]:
agent = qlearning.QLearningAgent(get_actions,
                                     epsilon=EPSILON,
                                     alpha=LEARNING_RATE,
                                     gamma=DISCOUNT_FACTOR,
                                     epsilon_decay=EPSILON_DECAY)

history = []

agent, history = train(agent, environment, history)

avg_reward = [np.mean(history[i*100:(i+1)*100]) for i in range(int(len(history)/100))]
f_reward = plt.figure(1)
plt.plot(np.linspace(0, len(history), len(avg_reward)), avg_reward)
plt.ylabel('Rewards')
f_reward.show()
plt.close()

Run = 0
Condition = True
while Condition == True:
    obs = np.copy(defaultstate)
    cur_state = discretize(obs)
    #   cur_state = np.copy(obs)
    done = False
    Run += 1
    if Run > NUM_EPISODES:
        Condition = False
    t = 0
    while not done:
        t = t+1
        action = agent.get_action(cur_state)
        observation, reward, done = environment(cur_state, action)
        next_state = discretize(observation)
#         if Run > 1995:
#             print('Action', action, 'Reward', reward)
#     next_state = np.copy(observation)
        if done:
            reward = FAIL_PENALTY
            agent.learn(cur_state, action, next_state, reward, done)
            print("Episode ", "finished after {} timesteps".format(t+1))
            history.append(t+1)
            break
        agent.learn(cur_state, action, next_state, reward, done)
        cur_state = next_state

Episode 1
Episode 101
Episode 201
Episode 301
Episode 401
Episode 501
Episode 601
Episode 701
Episode 801
Episode 901
Episode 1001
Episode 1101
Episode 1201
Episode 1301
Episode 1401
Episode 1501
Episode 1601
Episode 1701
Episode 1801
Episode 1901
Episode 2001
Episode 2101
Episode 2201
Episode 2301
Episode 2401
Episode 2501
Episode 2601
Episode 2701
Episode 2801
Episode 2901
Episode 3001
Episode 3101
Episode 3201
Episode 3301
Episode 3401
Episode 3501
Episode 3601
Episode 3701
Episode 3801
Episode 3901
Episode 4001
Episode 4101
Episode 4201
Episode 4301
Episode 4401
Episode 4501
Episode 4601
Episode 4701
Episode 4801
Episode 4901
Episode 5001
Episode 5101
Episode 5201
Episode 5301
Episode 5401
Episode 5501
Episode 5601
Episode 5701
Episode 5801
Episode 5901
Episode 6001
Episode 6101
Episode 6201
Episode 6301
Episode 6401
Episode 6501
Episode 6601
Episode 6701
Episode 6801
Episode 6901
Episode 7001
Episode 7101
Episode 7201
Episode 7301
Episode 7401
Episode 7501
Episode 7601
Episode 770

/home/deeplearning/.local/lib/python2.7/site-packages/matplotlib/figure.py:403: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "


('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 4 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 11 timesteps')
('Episode ', 'finished after 5 timesteps')
('Episode ', 'finished after 13 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 10 timesteps')
('Episode ', 'finished after 8 timesteps')
('Episode ', 'finished after 13 timesteps')
('Episode ', 'finished after 10 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 15 timesteps')
('Episode ', 'finished after 11 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 tim

('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 

('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 

...

('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 

('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 

('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 

('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 

('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 timesteps')
('Episode ', 'finished after 22 

In [7]:
obs = np.copy(defaultstate)
cur_state = discretize(obs)
#   cur_state = np.copy(obs)
done = False

t = 0
score = 0
while not done:
    t = t+1
    action = agent.get_action(cur_state)
    observation, reward, done = environment(cur_state, action)
    next_state = discretize(observation)
    score += reward
    print('State', observation, 'Action', action, 'Reward', reward)
    if done:
        reward = FAIL_PENALTY
        print("Episode ", "finished after {} timesteps".format(t+1))
        history.append(t+1)
        break
    cur_state = next_state
    
score

State [187.   5.   6.   0.   1.   0.   0.   0.   0.  10.   0.   0.   0.   0.
   0.   0.   0.   1.   0.   7.   1.   1.] Action 11 Reward 11776665.12
State [187.   5.   6.   0.   1.   0.   0.   0.   0.  10.   0.   0.   0.   0.
   0.   0.   0.   1.   0.   7.   2.   2.] Action 0 Reward 0
State [187.   5.   6.   0.   1.   0.   0.   0.   0.  10.   0.   0.   0.   0.
   0.   0.   0.   1.   0.   7.   0.   3.] Action 0 Reward 0
State [212.   5.   6.   0.   1.   0.   0.   1.   0.   9.   0.   0.   5.   1.
   0.   0.   0.   1.   0.   7.   1.   4.] Action 25 Reward 11881665.12
State [212.   5.   6.   0.   1.   0.   0.   1.   0.   9.   0.   0.   5.   1.
   0.   0.   0.   1.   0.   7.   2.   5.] Action 0 Reward 0
State [212.   5.   6.   0.   1.   0.   0.   1.   0.   9.   0.   0.   5.   1.
   0.   0.   0.   1.   0.   7.   0.   6.] Action 0 Reward 0
State [249.   5.   6.   0.   1.   1.   0.   1.   0.   8.   7.   0.   4.   1.
   0.   1.   0.   1.   0.   7.   1.   7.] Action 15 Reward 11316109.632
State [

60619162.985599995

In [7]:
action_space = np.matrix([0, 0])
for i in range(int(defaultstate[1])):
    for j in range(int(defaultstate[2])):
        action_space = np.vstack([action_space, (i+1, j+1)])
# action = np.ravel(action_space[action_no])


In [8]:
12, 20, 17, 1, 27

(12, 20, 17, 1, 27)

In [9]:
17, 30, 7, 21, 2

(17, 30, 7, 21, 2)

In [10]:
15, 20, 4, 30, 7

(15, 20, 4, 30, 7)

In [11]:
np.ravel(action_space[7])


array([2, 1])